In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import numpy as np
import tensorflow as tf
import pathlib
from tqdm import trange

from src.DataPreprocessor.data_preprocessor import DataPreprocessor, Mode
from src.LearningKeras.net_architecture import cnn_150x150x5,cnn_150x150x5_3class
from src.LearningKeras.train import KerasTrainer

np.random.seed(1)
tf.set_random_seed(2)

check data_visualisation for details

In [2]:
from src.DataPreprocessor.DataIOBackend.gdal_backend import GdalBackend
from src.DataPreprocessor.data_visualiser import DataVisualiser

dataiobackend = GdalBackend()
data_preprocessor = DataPreprocessor(data_dir="data/Region 1 - Lopukangri/",
                              backend=dataiobackend,
                              filename_prefix="tibet",
                              mode=Mode.TRAIN,
                              seed=1)

## NN

### Train

In [3]:
model_generator = lambda: cnn_150x150x5_3class()
ensemble_size = 3
batch_size = 10

trainer = KerasTrainer(model_generator=model_generator,
                       ensemble_size=ensemble_size,
                       data_preprocessor=data_preprocessor,
                       batch_size=batch_size)
train_generator = data_preprocessor.train_generator(batch_size=batch_size,
                                         class_probabilities=np.array([1./3, 1./3, 1./3]),
                                         patch_size=(150, 150),
                                         channels=np.array([0, 1, 2, 3, 4]))

In [4]:
history_arr = trainer.train(steps_per_epoch=100, epochs=5, train_generator=train_generator)

Epoch 1/5
100/100 [==============================] - 169s 2s/step - loss: 0.7560 - acc: 0.6500 - val_loss: 0.6243 - val_acc: 0.7400
Epoch 2/5
100/100 [==============================] - 161s 2s/step - loss: 0.5364 - acc: 0.7950 - val_loss: 0.5586 - val_acc: 0.6800
Epoch 3/5
100/100 [==============================] - 160s 2s/step - loss: 0.4590 - acc: 0.8120 - val_loss: 0.4295 - val_acc: 0.7600
Epoch 4/5
100/100 [==============================] - 160s 2s/step - loss: 0.3772 - acc: 0.8510 - val_loss: 0.4751 - val_acc: 0.7600
Epoch 5/5
100/100 [==============================] - 160s 2s/step - loss: 0.3736 - acc: 0.8430 - val_loss: 0.2626 - val_acc: 0.9400
Epoch 1/5
100/100 [==============================] - 163s 2s/step - loss: 0.9302 - acc: 0.5870 - val_loss: 0.6993 - val_acc: 0.7200
Epoch 2/5
100/100 [==============================] - 159s 2s/step - loss: 0.6174 - acc: 0.7430 - val_loss: 0.4612 - val_acc: 0.8400
Epoch 3/5
100/100 [==============================] - 160s 2s/step - loss: 0.

In [5]:
for i in range(ensemble_size):
    pathlib.Path('models_3class').mkdir(parents=True, exist_ok=True)
    trainer.models[i].save_weights('models_3class/model_{}.h5'.format(i))